In [1]:
import pandas as pd
import numpy as np
import folium

import requests
from urllib.parse import urlparse, quote

with open('C:/Users/Sum/Sumin/kakaoapikey.txt',mode='r') as key_file:
    kakao_key = key_file.read()

## 1.호텔

In [83]:
hotel = pd.read_csv('data/키워드_호텔_좋은순.csv')
hotel = hotel[:10]
hotel.head()

,hotel,index
0,페어필드 바이 메리어트 서울,1220
1,티마크 그랜드 호텔,570
2,머큐어 앰배서더 서울 홍대,402
3,더 스테이트 선유 호텔,399
4,[반짝특가] 신라스테이 구로,382


In [84]:
# 편의를 위해 변경
hotel.hotel[4] = '신라스테이 구로'

In [85]:
hotel.head()

,hotel,index
0,페어필드 바이 메리어트 서울,1220
1,티마크 그랜드 호텔,570
2,머큐어 앰배서더 서울 홍대,402
3,더 스테이트 선유 호텔,399
4,신라스테이 구로,382


In [86]:
local_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
local_url2 = "https://dapi.kakao.com/v2/local/search/address.json"

address = []
lng_list = []
lat_list = []

for i in hotel.index:
    #호텔 주소 가져오기
    url = f'{local_url}?query={quote(hotel["hotel"][i])}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try :
        add = result['documents'][0]['address_name']
    except : 
        add = '수동검색'
    address.append(add)
    
    #호텔 주소로 위경도 가져오기
    url = f'{local_url}?query={quote(add)}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try : 
        lng_list.append(result['documents'][0]['x'])
        lat_list.append(result['documents'][0]['y'])
    except Exception :
        lng_list.append('수동검색')
        lat_list.append('수동검색')

hotel['주소'] = address
hotel['위도'] = lat_list
hotel['경도'] = lng_list

hotel

,hotel,index,주소,위도,경도
0,페어필드 바이 메리어트 서울,1220,서울 영등포구 영등포동1가 143-3,37.51699304011284,126.90928157127733
1,티마크 그랜드 호텔,570,서울 중구 회현동1가 194-15,37.5583529991481,126.978464537203
2,머큐어 앰배서더 서울 홍대,402,서울 마포구 동교동 162-5,37.5553453560501,126.922394876232
3,더 스테이트 선유 호텔,399,서울 영등포구 양평동5가 62-3,37.5385857621445,126.893369463406
4,신라스테이 구로,382,서울 동작구 신대방동 698,37.4854935161792,126.90424971706
5,노보텔 스위트 앰배서더 서울 용산,315,서울 용산구 한강로3가 40-969,37.53178957663411,126.96320765235726
6,골드리버 호텔,302,서울 금천구 가산동 547-63,37.4783322835089,126.876980484436
7,영등포 GMS 호텔,291,서울 영등포구 영등포동3가 11-3,37.5174793690754,126.909004979609
8,노보텔 앰배서더 동대문 호텔&레지던스,271,서울 중구 을지로6가 58-5,37.5663088721492,127.004581922665
9,스탠포드 호텔 서울,256,서울 마포구 상암동 1587,37.5822850427227,126.886654718447


In [88]:
map = folium.Map(location=[hotel.위도.astype(float).mean(),hotel.경도.astype(float).mean()],zoom_start=12)

icon_url = 'data/location.png'
icon = folium.features.CustomIcon('data/location.png',icon_size=(45, 45))  

for i in hotel.index:
    folium.Marker(location = [hotel['위도'][i],hotel['경도'][i]],icon=folium.features.CustomIcon(icon_url,icon_size=(45, 45)),popup = folium.Popup(hotel['hotel'][i],min_width=170,max_width=170),tooltip=hotel['hotel'][i]).add_to(map)



map


In [89]:
map.save('hotel_top5.html')

## 2.호텔파티

In [41]:
party = pd.read_csv('data/final_top_100_hotel_호텔파티.csv')
party = party[:10][['hotel','star']]

In [42]:
party.head()

,hotel,star
0,노보텔 스위트 앰배서더 서울 용산,5
1,호텔 리베라,5
2,호텔 그라모스,5
3,더 스위트 캐슬,5
4,JW 메리어트 호텔 서울,5


In [45]:
local_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
local_url2 = "https://dapi.kakao.com/v2/local/search/address.json"

address = []
lng_list = []
lat_list = []

for i in party.index:
    #호텔 주소 가져오기
    url = f'{local_url}?query={quote(party["hotel"][i])}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try :
        add = result['documents'][0]['address_name']
    except : 
        add = '수동검색'
    address.append(add)
    
    #호텔 주소로 위경도 가져오기
    url = f'{local_url}?query={quote(add)}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try : 
        lng_list.append(result['documents'][0]['x'])
        lat_list.append(result['documents'][0]['y'])
    except Exception :
        lng_list.append('수동검색')
        lat_list.append('수동검색')
  
party['주소'] = address
party['위도'] = lat_list
party['경도'] = lng_list

party

,hotel,star,주소,위도,경도
0,노보텔 스위트 앰배서더 서울 용산,5,서울 용산구 한강로3가 40-969,37.53178957663411,126.96320765235726
1,호텔 리베라,5,서울 강남구 청담동 53-7,37.5238846595983,127.054264116487
2,호텔 그라모스,5,서울 강남구 역삼동 677-21,37.5025546192533,127.039872234482
3,더 스위트 캐슬,5,서울 강남구 역삼동 603-5,37.50575978819907,127.02969961565248
4,JW 메리어트 호텔 서울,5,서울 서초구 반포동 19-3,37.505305732081965,127.00432032924516
5,노보텔 앰배서더 동대문 호텔&레지던스,5,서울 중구 을지로6가 58-5,37.5663088721492,127.004581922665
6,레스케이프,5,서울 중구 회현동1가 210,37.559682151965866,126.97948729970663
7,티마크 그랜드 호텔,4,서울 중구 회현동1가 194-15,37.5583529991481,126.978464537203
8,알로프트 서울 명동,5,서울 중구 명동2가 104,37.562335985428,126.981922286996
9,밀레니엄 힐튼 서울,4,서울 중구 남대문로5가 395,37.555326869140345,126.97548669011121


In [69]:
map = folium.Map(location=[party.위도.astype(float).mean(),party.경도.astype(float).mean()],zoom_start=12)

icon_url = 'data/location.png'
icon = folium.features.CustomIcon('data/location.png',icon_size=(45, 45))  

for i in party.index:
    folium.Marker(location = [party['위도'][i],party['경도'][i]],icon=folium.features.CustomIcon(icon_url,icon_size=(45, 45)),popup = folium.Popup(party['hotel'][i],min_width=170,max_width=170),tooltip=party['hotel'][i]).add_to(map)



map

In [70]:
map.save('party_top5.html')

### 3. 호캉스

In [9]:
hotel = pd.read_csv('data/호캉스_top_100.csv')
hotel = hotel[:12]
hotel.head()

,hotel,index
0,페어필드 바이 메리어트 서울,12944
1,영등포 GMS 호텔,5960
2,신라스테이 구로,5031
3,[반짝특가] 신라스테이 구로,2348
4,스탠포드 호텔 서울,2329


In [10]:
# 편의를 위해 변경
hotel.hotel[3] = '신라스테이 구로'

In [11]:
hotel.drop(index=3,axis=0,inplace=True)

<bound method DataFrame.reset_index of               hotel  index
0   페어필드 바이 메리어트 서울  12944
1        영등포 GMS 호텔   5960
2          신라스테이 구로   5031
4        스탠포드 호텔 서울   2329
5             호텔 데님   1818
6      더 스테이트 선유 호텔   1782
7    머큐어 앰배서더 서울 홍대   1721
8            호텔 리베라   1672
9           스탠다드 호텔   1664
10  비지비 미디 나이아가라 호텔   1659
11    호텔 더 디자이너스 종로   1659>

In [17]:
hotel.reset_index()
hotel

,hotel,index
0,페어필드 바이 메리어트 서울,12944
1,영등포 GMS 호텔,5960
2,신라스테이 구로,5031
4,스탠포드 호텔 서울,2329
5,호텔 데님,1818
6,더 스테이트 선유 호텔,1782
7,머큐어 앰배서더 서울 홍대,1721
8,호텔 리베라,1672
9,스탠다드 호텔,1664
10,비지비 미디 나이아가라 호텔,1659


In [18]:
local_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
local_url2 = "https://dapi.kakao.com/v2/local/search/address.json"

address = []
lng_list = []
lat_list = []

for i in hotel.index:
    #호텔 주소 가져오기
    url = f'{local_url}?query={quote(hotel["hotel"][i])}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try :
        add = result['documents'][0]['address_name']
    except : 
        add = '수동검색'
    address.append(add)
    
    #호텔 주소로 위경도 가져오기
    url = f'{local_url}?query={quote(add)}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try : 
        lng_list.append(result['documents'][0]['x'])
        lat_list.append(result['documents'][0]['y'])
    except Exception :
        lng_list.append('수동검색')
        lat_list.append('수동검색')

hotel['주소'] = address
hotel['위도'] = lat_list
hotel['경도'] = lng_list

hotel

,hotel,index,주소,위도,경도
0,페어필드 바이 메리어트 서울,12944,서울 영등포구 영등포동1가 143-3,37.51699304011284,126.90928157127733
1,영등포 GMS 호텔,5960,서울 영등포구 영등포동3가 11-3,37.5174793690754,126.909004979609
2,신라스테이 구로,5031,서울 동작구 신대방동 698,37.4854935161792,126.90424971706
4,스탠포드 호텔 서울,2329,서울 마포구 상암동 1587,37.5822850427227,126.886654718447
5,호텔 데님,1818,서울 강남구 개포동 1229-14,37.4761309913882,127.046641410362
6,더 스테이트 선유 호텔,1782,서울 영등포구 양평동5가 62-3,37.5385857621445,126.893369463406
7,머큐어 앰배서더 서울 홍대,1721,서울 마포구 동교동 162-5,37.5553453560501,126.922394876232
8,호텔 리베라,1672,서울 강남구 청담동 53-7,37.5238846595983,127.054264116487
9,스탠다드 호텔,1664,서울 강서구 등촌동 505-7,37.5509960479657,126.863539186793
10,비지비 미디 나이아가라 호텔,1659,서울 강서구 염창동 259-2,37.54915919048632,126.87864876808253


In [19]:
map = folium.Map(location=[hotel.위도.astype(float).mean(),hotel.경도.astype(float).mean()],zoom_start=12)

icon_url = 'data/location.png'
icon = folium.features.CustomIcon('data/location.png',icon_size=(45, 45))  

for i in hotel.index:
    folium.Marker(location = [hotel['위도'][i],hotel['경도'][i]],icon=folium.features.CustomIcon(icon_url,icon_size=(45, 45)),popup = folium.Popup(hotel['hotel'][i],min_width=170,max_width=170),tooltip=hotel['hotel'][i]).add_to(map)

map

In [20]:
map.save('hokangs_top5.html')

### 4.가성비호텔

In [79]:
hotel = pd.read_csv('data/가성비호텔_top100.csv')
hotel = hotel[:10]
hotel.head()

,hotel,index
0,페어필드 바이 메리어트 서울,12944
1,신라스테이 구로,7379
2,영등포 GMS 호텔,5960
3,스탠포드 호텔 서울,2329
4,호텔 데님,1818


In [80]:
local_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
local_url2 = "https://dapi.kakao.com/v2/local/search/address.json"

address = []
lng_list = []
lat_list = []

for i in hotel.index:
    #호텔 주소 가져오기
    url = f'{local_url}?query={quote(hotel["hotel"][i])}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try :
        add = result['documents'][0]['address_name']
    except : 
        add = '수동검색'
    address.append(add)
    
    #호텔 주소로 위경도 가져오기
    url = f'{local_url}?query={quote(add)}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try : 
        lng_list.append(result['documents'][0]['x'])
        lat_list.append(result['documents'][0]['y'])
    except Exception :
        lng_list.append('수동검색')
        lat_list.append('수동검색')

hotel['주소'] = address
hotel['위도'] = lat_list
hotel['경도'] = lng_list

hotel

,hotel,index,주소,위도,경도
0,페어필드 바이 메리어트 서울,12944,서울 영등포구 영등포동1가 143-3,37.51699304011284,126.90928157127733
1,신라스테이 구로,7379,서울 동작구 신대방동 698,37.4854935161792,126.90424971706
2,영등포 GMS 호텔,5960,서울 영등포구 영등포동3가 11-3,37.5174793690754,126.909004979609
3,스탠포드 호텔 서울,2329,서울 마포구 상암동 1587,37.5822850427227,126.886654718447
4,호텔 데님,1818,서울 강남구 개포동 1229-14,37.4761309913882,127.046641410362
5,더 스테이트 선유 호텔,1782,서울 영등포구 양평동5가 62-3,37.5385857621445,126.893369463406
6,머큐어 앰배서더 서울 홍대,1721,서울 마포구 동교동 162-5,37.5553453560501,126.922394876232
7,호텔 리베라,1672,서울 강남구 청담동 53-7,37.5238846595983,127.054264116487
8,스탠다드 호텔,1664,서울 강서구 등촌동 505-7,37.5509960479657,126.863539186793
9,비지비 미디 나이아가라 호텔,1659,서울 강서구 염창동 259-2,37.54915919048632,126.87864876808253


In [81]:
map = folium.Map(location=[hotel.위도.astype(float).mean(),hotel.경도.astype(float).mean()],zoom_start=12)

icon_url = 'data/location.png'
icon = folium.features.CustomIcon('data/location.png',icon_size=(45, 45))  

for i in hotel.index:
    folium.Marker(location = [hotel['위도'][i],hotel['경도'][i]],icon=folium.features.CustomIcon(icon_url,icon_size=(45, 45)),popup = folium.Popup(hotel['hotel'][i],min_width=170,max_width=170),tooltip=hotel['hotel'][i]).add_to(map)

map

In [82]:
map.save('reasonable_top5.html')